# Static file management with SourceLoader

Data pipelines usually interact with external systems such as SQL databases. 



In [1]:
from pathlib import Path
import tempfile
import shutil

import pandas as pd
from sklearn import datasets
from IPython.display import display, Markdown

from ploomber import DAG, SourceLoader
from ploomber.tasks import PythonCallable, NotebookRunner, SQLUpload, SQLScript
from ploomber.products import File, SQLiteRelation
from ploomber.clients import SQLAlchemyClient

In [11]:
# initialize a temporary directory
tmp_dir = Path(tempfile.mkdtemp())
tmp_dir_static = tmp_dir / 'static'
tmp_dir_static.mkdir()

report_py = """
# static/report.py

# +
# This file is in jupytext light format
import seaborn as sns
import pandas as pd
# -

# + tags=['parameters']
# papermill will add the parameters below this cell
upstream = None
product = None
# -

# +
path = upstream['raw']
df = pd.read_parquet(path)
# -

# ## AGE distribution

# +
_ = sns.distplot(df.AGE)
# -

# ## Price distribution

# +
_ = sns.distplot(df.price)
# -
"""

clean_table_sql = """
-- static/clean_table.sql

DROP TABLE IF EXISTS {{product}};

CREATE TABLE {{product}}
AS SELECT * FROM {{upstream["raw_table"]}}
WHERE AGE < 100
"""

(tmp_dir_static / 'report.py').write_text(report_py)
_ = (tmp_dir_static / 'clean_table.sql').write_text(clean_table_sql)

def display_file(file, syntax):
    s = """
```{}
{}
```
""".format(syntax, file)
    return display(Markdown(s))

Say our working environment has a `static/` folder with a SQL and a Python script.

In [3]:
! tree $tmp_dir

/var/folders/3h/_lvh_w_x5g30rrjzb_xnn2j80000gq/T/tmpmup5ltkr
└── static
    ├── clean_table.sql
    └── report.py

1 directory, 2 files


### Content of `static/report.py`

In [12]:
display_file(report_py, 'python')


```python

# static/report.py

# +
# This file is in jupytext light format
import seaborn as sns
import pandas as pd
# -

# + tags=['parameters']
# papermill will add the parameters below this cell
upstream = None
product = None
# -

# +
path = upstream['raw']
df = pd.read_parquet(path)
# -

# ## AGE distribution

# +
_ = sns.distplot(df.AGE)
# -

# ## Price distribution

# +
_ = sns.distplot(df.price)
# -

```


### Content of `static/create_table.sql`

In [13]:
display_file(clean_table_sql, 'sql')


```sql

-- static/clean_table.sql

DROP TABLE IF EXISTS {{product}};

CREATE TABLE {{product}}
AS SELECT * FROM {{upstream["raw_table"]}}
WHERE AGE < 100

```


### Pipeline declaration

In [31]:
def _get_data(product):
    data = datasets.load_boston()
    df = pd.DataFrame(data.data)
    df.columns = data.feature_names
    df['price'] = data.target
    df.to_parquet(str(product))

dag = DAG()

client = SQLAlchemyClient('sqlite:///my_db.db')
dag.clients[SQLUpload] = client
dag.clients[SQLiteRelation] = client
dag.clients[SQLScript] = client

# initialize SourceLoader in our static directory
loader = SourceLoader(path=tmp_dir_static)

get_data = PythonCallable(_get_data,
                          product=File(tmp_dir / 'raw.parquet'),
                          dag=dag,
                          name='raw')

# if we do not pass a name, the filename will be used as default
report = NotebookRunner(loader['report.py'],
                        product=File(tmp_dir / 'report.html'),
                        dag=dag,
                        kernelspec_name='python3',
                        ext_in='py')

raw_table = SQLUpload(source='{{upstream["raw"]}}',
                     product=SQLiteRelation(('raw', 'table')),
                     dag=dag,
                     name='raw_table')

# same here, no need to pass a name
clean_table = SQLScript(loader['clean_table.sql'],
                        product=SQLiteRelation(('clean', 'table')),
                        dag=dag)

get_data >> report
get_data >> raw_table >> clean_table

SQLScript: clean_table.sql -> SQLiteRelation(clean)

### Pipeline status

In [32]:
# Location column automatically adds the path to the source code to our static files
dag.status()

name,Last updated,Outdated dependencies,Outdated code,Product,Doc (short),Location
raw,Has not been run,False,True,/var/folders/3h/_lvh_w_x5g30rrjzb_xnn2j80000gq/T/tmpf8q2kzwo/raw.parquet,,:1
report.py,Has not been run,True,True,/var/folders/3h/_lvh_w_x5g30rrjzb_xnn2j80000gq/T/tmpf8q2kzwo/report.html,,/var/folders/3h/_lvh_w_x5g30rrjzb_xnn2j80000gq/T/tmpf8q2kzwo/static/report.py
raw_table,"23 minutes ago (Mar 14, 20 at 17:24)",True,True,raw,,
clean_table.sql,"23 minutes ago (Mar 14, 20 at 17:24)",True,False,clean,,/var/folders/3h/_lvh_w_x5g30rrjzb_xnn2j80000gq/T/tmpf8q2kzwo/static/clean_table.sql


In [33]:
dag.build()

name,Ran?,Elapsed (s),Percentage
raw,True,0.115526,3.43378
raw_table,True,0.034885,1.03689
clean_table.sql,True,0.005273,0.156729
report.py,True,3.20872,95.3726


## Advanced jinja2 features

`SourceLoader` initializes a proper jinja2 environment, so you can use features such as [macros](https://jinja.palletsprojects.com/en/2.11.x/templates/#macros), this is very useful to maximize SQL code reusability.

In [34]:
shutil.rmtree(tmp_dir)